In [1]:
#Imports
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [18]:
#Merging data
years = [2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014]
years= [str(i) for i in years]
data={}

for i in years:
    data[i] = pd.read_csv(i+'plays.csv')
    data[i]['year'] = int(i)

fulldf = pd.concat(data.values())
fulldf = fulldf.reset_index(drop=True)
fullplays = len(fulldf)

In [19]:
# create columns for half and if the team with the ball is the home team, as well as time left in half
fulldf['is1stHalf'] = fulldf['quarter'] < 3
fulldf['is2ndHalf'] = (fulldf['quarter'] > 2) & (fulldf['quarter'] < 5)
fulldf['half'] = 2 - (fulldf['is1stHalf'])
for i,row in fulldf.iterrows():
    if row['quarter'] == 5:
        fulldf.loc[i,'half'] = 'OT'

fulldf['isHome'] = fulldf['tm'] == fulldf['home']
fulldf['secsElapsedInHalf'] = np.remainder(fulldf['secsElapsedInGame'],1800)
fulldf['isLast3minHalf'] = fulldf['secsElapsedinHalf'] >= 1620

KeyboardInterrupt: 

In [ ]:
# create columns for home and away timeouts remaining in the half
for x in ['hm_TO_left','aw_TO_left']:
    fulldf[x] = [3 for i in xrange(fullplays)]
for i,row in fulldf.iterrows():
    if row['half'] == 'OT':
        fulldf.loc[i,'hm_TO_left'] = 2
        fulldf.loc[i,'aw_TO_left'] = 2
    
for i,row in fulldf.iterrows():
    if row['isTimeout']:
        r = fulldf[(fulldf['bsID']==row['bsID']) & (fulldf['half']==row['half'])]
        l = range(i+1,r.index[-1])
        if row['timeoutTeam'] == row['home']:
            fulldf.loc[l,'hm_TO_left'] = r.loc[l,'hm_TO_left'] - 1
        elif row['timeoutTeam'] == row['away']:
            fulldf.loc[l,'aw_TO_left'] = r.loc[l,'aw_TO_left'] - 1

In [ ]:
# only keeep run and pass plays from fulldf in the dataframe df
dfrunpass = fulldf[(fulldf['isPass']== True) | (fulldf['isRun'] == True)]
df = dfrunpass.reset_index(drop=True)

In [ ]:
# Create feature columns based on given home/away information (tm = team with ball, opp = opponent)
plays = len(df)
for x in ['tm_score','opp_score','tm_winprob','tm_TO_left','opp_TO_left']:
    df[x] = [0 for i in xrange(plays)]
    
for i,row in df.iterrows():
    if row['isHome']:
        df.loc[i,'tm_score'] = row['pbp_score_hm']
        df.loc[i,'opp_score'] = row['pbp_score_aw']
        df.loc[i,'tm_winprob'] = row['home_wp']
        df.loc[i,'tm_TO_left'] = row['hm_TO_left']
        df.loc[i,'opp_TO_left'] = row['aw_TO_left']
    else:
        df.loc[i,'tm_score'] = row['pbp_score_aw']
        df.loc[i,'opp_score'] = row['pbp_score_hm']
        df.loc[i,'tm_winprob'] = 100.0 - row['home_wp']
        df.loc[i,'tm_TO_left'] = row['aw_TO_left']
        df.loc[i,'opp_TO_left'] = row['hm_TO_left']

# create scoring marging column (team with ball - opponent) and indicators if the team with the ball is in FG Range and/or RedZone
df['margin'] = df['tm_score'] - df['opp_score']
df['isFGRange'] = df['distToGoal'] <= 38
df['isRedZone'] = df['distToGoal'] <= 20

In [ ]:
df[df['year']<2009].to_csv('0208plays.csv', index=False, encoding = 'utf-8')
df[df['year']>2008].to_csv('0914plays.csv', index=False, encoding = 'utf-8')

In [ ]:
df = pd.concat([pd.read('0208.csv'),pd.read('0208.csv')])
df = 